## 1. Configuration

In [1]:
# ipython kernel install --name ".venv" --user
import os
from openai import AzureOpenAI
from string import Template
import json
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep

In [2]:
# Openai configuration
openai_api_key=os.getenv("OPENAI_API_KEY")
azure_endpoint=os.getenv("OPENAI_ENDPOINT")
openai_api_version=os.getenv("OPENAI_API_VERSION")
openai_model=os.getenv("OPENAI_MODEL")

client = AzureOpenAI(  
    api_key=openai_api_key,
    azure_endpoint=azure_endpoint,  
    api_version=openai_api_version,  
)

## 2. Helper Functions

In [3]:
# Function to call OpenAi
def process_gpt(file_prompt, system_msg):
    completion = client.chat.completions.create(
        model=openai_model,
        max_tokens=15000,
        temperature=0,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": file_prompt},
        ],
    )
    nlp_results = completion.choices[0].message.content
    return nlp_results



### 3. Running

In [ ]:
file_prompt = """
can you give me a neet json list of the following questions, also remove prefixes of letters, numbers, hyphens and small descriptions
if there are typos in the questions correct them

only return valid json, DO NOT ADD the word json at the beginning or ```
only return valid json, DO NOT ADD the word json at the beginning or ```
only return valid json, DO NOT ADD the word json at the beginning or ```

for example the second question should be placed transformed as follows and end up in the json list like so:
 "1. General Information. A. What is the location of Unigestion Head Office? 
-> What is the location of Unigestion Head Office? (Relevant part of question to be extracted)
-> [..., "What is the location of Unigestion Head Office? ",... ] (Embedded in json List)

questions:
Given Unigestion’s stated commitment to ESG integration, how does the exclusion criteria based on the UN Principles for Responsible Investment influence their preferential access to off-market deals in secondary markets?
"1. General Information. A. What is the location of Unigestion Head Office?
What is Unigestion full postal address?
who clarifies Unigestions goals (e.g., high returns, portfolio diversification, risk tolerance)
How does Unigestion ensure consistency in valuations when market conditions fluctuate rapidly?
Unigestion highlights a 20+ year history in the global secondaries market. How has the acquisition of Akina in 2017 enhanced their positioning in mid-market opportunities, and what are the quantifiable impacts on fund performance metrics?
What metrics does Unigestion use to measure the alpha performance of its mid-market secondary investments?
How does the direct investment mindset in USEC VI ensure consistent due diligence across secondary funds, and how does this align with governance practices applied to other fund types?
How does Unigestion's commitment to the UN Principles for Responsible Investment influence its allocation strategy in USEC VI?
B. Which Year was Unigestion Established?
32. Have there been any corporate changes over the past five years at Unigestion? If yes, please provide details.
6. What is the Investment Period of UNIGESTION SECONDARY VI Fund?
"C. Who has formal oversight and accountability for ESG / RI at Unigestion? Choices from: Organisation's board, Chief-level of staff (e.g. CEO, CIO, COO), Investment Committee, Responsible Investment Committee, Head of ESG/Responsible Investment, Head of other department (e.g. CFO, Head of Risk), None of the above roles have oversight and accountability for RI, Other"
B. In which year was your responsible investment policy introduced?
29. Please describe, using a diagram, the organisational structure of Unigestion, highlighting the team that is responsible for Private Equity Secondaries and more specifically the team that would be responsible for the presented mandate. Please specify if this team is also covering other strategies
"Strategy Vehicle information 1. UNIGESTION SECONDARY VI Fund 2. If different from the GP, please provide the AIFM for UNIGESTION SECONDARY VI Fund"
36. Please provide an overview of the knowledge transfer services from Unigestion that would be available to this client.
8. What is the Target of UNIGESTION SECONDARY VI Fund Size in euros?
26. What is the Anticipated number of underlying fundsof UNIGESTION SECONDARY VI Fund?
30. What are Unigestion's long-term strategic objectives?
13. What is the Final close date of UNIGESTION SECONDARY VI Fund ?
14. What is the Base Currency of UNIGESTION SECONDARY VI Fund ?
23. What is the Target number of transactions of UNIGESTION SECONDARY VI Fund ?
5. Please discuss Unigestion's approach to leverage and any challenges in this regard given current market conditions.
"18. What is the Target IRR (Gross) of UNIGESTION SECONDARY VI Fund ? "
2. Please outline Unigestion's investment strategy for this program and discuss how Unigestion thinking in this space has evolved over time.
4. Please provide information on how Artificial Intelligence (if at all) is used in the process at Unigestion?
30. At what stage of funded commitments do you generally enter a secondary transaction?
"Investment Process Investment Strategy 1. Please describe Unigestion's investment process. Discuss their investment philosophy and rationale for investing in asset class, sourcing/research, fund/investment selection and portfolio construction. 500 words max."
13. Please provide a recent case-study example to demonstrate Unigestion due diligence process to assess an underlying fund’s ESG capabilities?
15. Please provide an example of a recent due diligence report from Unigestion.
11. Describe Unigestion's overall philosophy to ESG investing within private equity.Stage 2
24. What is tge Typical transaction size for the proposed program of UNIGESTION SECONDARY VI Fund ?. Please also provide an historical range for past vehicles in the strategy.
12. Please describe how Unigestion assess ESG policies and practices of underlying funds and how this is factored into the investment process including details of any third-party data used. If applicable, please provide a sample of Unigestion questionnaire for the underlying funds.
21. Please provide a list of Investment Committee members from Unigestion.
22. Please provide resumes for all Investment Committee members at Unigestion
"19. How are ESG risks and opportunities reported, considered, and documented by the ultimate decision-making body, such as the investment committee at Unigestion? If applicable, please also describe any look through processes for underlying portfolios relating to both ESG and climate change."
27. Has Unigestion ever invested in, or approved for investment, a company which has had serious issues or required a termination? Please provide details. Did such an event result in any modification of the investment or due diligence process?
"8. Please provide a current deal log at Unigestion. Company/fund names may be anonymised if required. A screenshot of a deal log table can be copy and pasted into the below question."
20. Please provide 2 examples of ESG related issues that have affected Unigestion's strategy, their impact and the activities undertaken in response.
16. Please discuss Unigestion's approach to transaction pricing and how this has changed over time. Please provide some views on the expected pricing environment during the life of the program.
14. Describe the potential risk factors that are specific to the proposed portfolio and specific safeguards contemplated to preserve investor capital at Unigestion.
29. Describe Unigestion's engagement process with underlying funds including who is responsible for this process. Please provide 2 examples of how you have engaged with underlying and the eventual outcome of this engagement.Stage 2
35. How often and how soon after period end can Unigestion produce full audited reports?
34. Please describe the depth, frequency and timeliness of the performance reporting / portfolio overview that Unigestion provide to clients. Are in-house resources dedicated to this area? Please indicate the individuals that would be responsible for ongoing client service for this mandate.
28. Please provide details of Unigestion's ongoing monitoring of underlying funds and companies including how they often review their overall assessment of the underlying investment.
42. Have there been any adverse ESG events across the Unigestion portfolio, that have been communicated to investors? Please describe below.
"39. Do Unigestion provide online reporting? Choice from: Yes OR No"
"Exit 30. Please detail Unigestion's approach to exiting investments."
40. What is Unigestion's approach to capital recycling? Please describe any limits which are placed on the ability to recycle capital in this way (e.g. time limit, commitment cap etc.). Please also indicate the extent to which capital recycling has been used in prior funds/ SMAs. What has been the impact on returns (IRR and MOIC) historically? What is the expected impact?
"B. Are these individuals a centralised resource for Unigestion, asset class specific, or are they dedicated to ESG strategies? Choices from: Centralised resource for firm, Divided based on asset classes, Dedicated to strategies with explicit ESG focus, Both centralised and dedicated strategy resources, Not applicable, Other (please comment)"
C. Is the policy board-approved? Choice from: Yes OR No
"4. Firm-level ESG resourcing A. Does Unigestion employ any full-time professionals dedicated to ESG? Choice from: Yes OR No"
38. Describe Unigestion's approach to disclosing and following up on material ESG incidents to your investors.
"ESG at firm level 3. Firm-level ESG policy A. Does Unigestion have a formal policy covering their approach to responsible investment? Choices from: Yes, No (but in development), No"
36. What key indicators are published by Unigestion for this strategy at portfolio level?
"Firm-level ESG DDQ 1. Have Unigestion completed bfinance's firm-level ESG DDQ? Choice from: Yes OR No"
"5. Firm-level exclusions A. Please indicate the sectors Unigestion have decided to exclude at firm level: Choices from: Controversial weapons (e.g. cluster munitions, anti-personnel mines, white phosphorus), Conventional weapons, Nuclear weapons, Fossil fuels, Nuclear energy, Palm oil, Tobacco, Gambling, Alcohol, Animal testing, Adult entertainment, Other (please list)"
How many percent is multi asset funds & mandates in Unigestions asset
2. If yes, please confirm your responses are up to date for this search? Choice from: Yes OR No
10. If applicable what SFDR article is the proposed strategy categorised as?
"Post Investment Period 3. Please indicate the management fee (in %) Unigestion will charge the client per annum based on a EUR 30m investment"
11. If applicable, can you detail the process engaged on an on-going basis to make sure the fund at Unigestion is fully compliant with the above?
"ESG Exclusions (Strategy-Level) 12. Does the strategy apply any ESG-related exclusions (in addition to any exclusions that are applied at firm-level)? Choice from: Yes OR No"
"C. Please list any other ESG/RI related collaborative organisations/initiatives Unigestion participate in: Examples could include: EUROSIF, GIIN, GRESB, GRI, UK SIF, UN Global Compact, Diversity Project, ESG Convergence Project Other"
19. Does Unigestion provide carbon reporting for this mandate?
How does Unigestion’s ability to review 1,500 deals annually across Europe, North America, and Asia compare to their deal execution rate in 2022? How does this translate into their focus on mid-market alpha opportunities?
4. The management fee is based on what at Unigestion? Choices from: Committed Capital, Invested Capital, NAV, Other - If selected please describe what the management fee is based on if not one of the above options?
"Further ESG Questions 20. Has a potential investment ever been rejected solely by Unigestion due to ESG related issues? Please provide at least one (1) example"
What is the maximum leverage allowed across fund levels in USEC VI, and how is this limitation justified against its stated risk strategy?
16. Do Unigestion explicitly consider environmental (including climate change) risks in the management of the portfolio?
2. The management fee at Unigestion is based on? Choices from: Committed Capital, Invested Capital, NAV, Other - If selected please describe what the management fee is based on if not one of the above options?
who from Unigestion has built private equity teams
can 1000 deals be doneper year if clients source unique opportunities from Unigestions network of over 500 GPs
did de Dardel ever work in a company that has a business unit in agriculture business
Describe the impact of ESG principles on fund performance metrics and allocation strategies as applied by Unigestion.
"Management Fees During the Investment Period 1. Please indicate the management fee (in %) Unigestion will charge the client per annum based on a EUR 30m investment"
What specific regulatory differences affect the structuring of Unigestion funds domiciled in Luxembourg versus those in Switzerland?
26. Post investment, how do Unigestion add value to their portfolio?
How does the investment approach for secondary funds (USEC VI) differ in structure and returns from the direct investment approach described in Unigestion’s policies?
"Post-Investment Monitoring 25. Describe the Fund Manager’s philosophy to ensure appropriate oversight control of the portfolio at Unigestion."
23. Are there any managed accounts or otherwise that will invest alongside this program at Unigestion? If so, please provide some details on their allocation policy.
25. what is the Anticipated number of underlying GPs of UNIGESTION SECONDARY VI Fund
How does the Valuation Committee ensure consistency in applying IPEV guidelines for direct investments, and what happens in case of methodology changes?
18. Does Unigestion provide ESG-related reporting on the proposed fund / strategy?
"European Bank Investor Experience 4. Please provide details on Unigestion's experience working with European and Bank investors (in general)"
Please provide in EURm
2. Please briefly explain Unigestion's experience in managing Private Equity Secondaries strategies. When did Unigestion first start managing strategies in this space?
35. How does Unigestion deal with any conflicts of interest both within the organisation and in relation to third parties?
"3. UNIGESTION SECONDARY VI Fund Structure(s) Please make sure to provide all structures available of UNIGESTION SECONDARY VI Fund"
32. What is Unigestion's approach to incorporating ESG considerations into preparations for exit and ensuring continued positive impact in the post exit phase?
27. Limits (Sector, Geography)
5. What are the UNIGESTION SECONDARY VI Fund's Terms?
41. Please describe Unigestion ESG reporting capabilities for this fund, including frequency, metrics reported and level of transparency. Please also describe Unigestion's approach to ensuring data availability, quality and reliability.
7. Please discuss Unigestion's historical deal pipeline in terms of the conversion funds seen per annum, those taken through the various stages of diligence and those that closed.
37. Please provide details such as metrics, frequency etc. of Unigestion's climate related reporting including a sample report. Please comment on how you address areas where data is not available
"8. Principles for Responsible Investment and industry collaborations A. Is Unigestion currently a signatory to the Principles for Responsible Investment (PRI)? Choices from: Yes, No (but intending to become a signatory in 6 months), No (but actively considering becoming a signatory), No"
B. If yes, when did Unigestion become a signatory?
31. How do you see Unigestion evolving in terms of ownership structure, client base, product offering, etc.?
21. Please detail all internal resourcing utilised for ESG analysis at Unigestion, with specific reference to dedicated in-house roles.
29. Please confirm that you do not use any leverage at the transaction level.
31. Does Unigestion take an active approach to winding up investments?
24. Please provide an example of an investment from Unigestion that they have reviewed and conducted due diligence on but that did not result in an investment.
34. Has Unigestion changed auditors in the past five years? If so, please explain why.
33. Are you aware of any current (or potential) changes to Unigestion's organisational structure or ownership?
"4. UNIGESTION SECONDARY VI Fund Domicile(s) Please make sure to provide all domiciles available of UNIGESTION SECONDARY VI Fund"
B. Have you set any organisation-wide targets on climate change? Choices from: Reducing carbon intensity of portfolios, Investing in low-carbon, energy-efficient climate adaptation opportunities in different asset classes, Aligning the entire group-wide portfolio with net zero, Reducing exposure to assets with significant climate transition risks, Aligning a proportion of portfolios with net zero, No, we have not set any climate-related targets, Other (please state)
9. Please provide a brief description and history of Unigestion.
3. Please provide information on the IT systems in place to support the investment process at Unigestion. Please make sure to identify the tools and systems that are proprietary / developed internally (if any).
17. Does the investment team at Unigestion actively engage with investee companies/underlying managers on ESG-related topics?
"19. What is the Target IRR (Net) of UNIGESTION SECONDARY VI Fund ? "
17. What is Unigestion's general approach to the use of leverage at both the company level and at the fund level?
21. What is the Target TVPI (Net) of UNIGESTION SECONDARY VI Fund ?
What specific attributes make USEC VI’s portfolio of small secondary transactions ‘unlikely to be replicated elsewhere,’ and how does this impact Unigestion’s ability to maintain consistent alpha performance across cycles?
"Asset Class Experience Private Equity Experience 1. Please briefly explain Unigestion's experience in managing Private Equity strategies. When did the firm first begin investing in the asset class?"
22. Are any external resources or organisations used for ESG analysis at Unigestion?
15. If applicable, please list any specific ESG-related KPIs measured and / or SDG alignment within the strategy?
14. Does the investment team at Unigestion explicitly consider ESG / RI as part of the investment process of the proposed strategy at Unigestion?
B. What is the Main activities of Unigestion?
10. What are the key qualitative and quantitative criteria addressed during due diligence at Unigestion?
"6. Firm-level climate change A. Do you have a climate-related investment policy? Choices from: Yes (standalone policy), Yes (specifically addressed within ESG/RI policy), No (but in development), No"
28. Please provide the details of the ownership structure at Unigestion, i.e. details of shareholders with % ownership for each individual or group
"Valuation and Reporting 33. Describe Unigestion's valuation process including the name and the role of any independent, external valuers. With what frequency and according to which accounting standards are investments valued? How frequently do Unigestion receive valuations from their investments?"
"Decision-making 18. Please describe the investment committee process and the voting procedures at Unigestion. Are investment decisions made unanimously / through majority (specify %) at Unigestion? Does any member of the Investment Committee have a veto right at Unigestion?"
"Due Diligence 9. Please describe Unigestion's due diligence process for the strategy, from initial screening to final investment decision."
10. What is the Fundraising start date of of UNIGESTION SECONDARY VI Fund ?
"Investment Sourcing 6. How are investments sourced and selected at Unigestion and how is this approach differentiated to Unigestion's competitors?"
13. If yes, please list these additional exclusions.
7. What is the Minimum Ticket Size in euros of UNIGESTION SECONDARY VI Fund?
How does Unigestion's strategy for small, non-auctioned transactions mitigate competition risk in the mid-market?
If Unigestion targets 328 primary, 353 secondary, and 350 direct opportunities annually, why is the mid-market segment particularly advantageous for secondary investments, as stated in the USEC VI fund strategy?
How has the acquisition of Akina in 2017 influenced Unigestion’s ability to scale its private equity strategies?
12. What is the Next close date of of UNIGESTION SECONDARY VI Fund ?
9. What is the Hard Cap of UNIGESTION SECONDARY VI Fund Size in euros?
"22. Deployment pace State the number deployed per year / months by UNIGESTION SECONDARY VI Fund ?"
C. With which entity would the client be contracting at Unigestion?
"Strategy Information 16. Please detail the proposed portfolio’s investment strategy (i.e.: LP Stakes, GP-led transactions etc.). Where do you see the most attractive opportunities in the current market environment? How does this fit with the proposed portfolio construction? Are there any strategies you would exclude (e.g. venture, distressed etc.)? 300 words max."
28. Will you be using subscription line financing at Unigestion? If so, what is the purpose and intention of using the line and what are the terms? Provide details including % of commitments. What has been the impact on returns (IRR and MOIC) historically? What is the expected impact? How will you report on this to LPs?
11. What is the Amount raised so far for UNIGESTION SECONDARY VI Fund Size in euros?
"Proposed Strategy 9. Does the proposed mandate fall into scope of achieving net zero at Unigestion? Choice from: Yes OR No"
"5. Firm Level - Accounting Contact Information What is Unigestion entity to be invoiced? "
17. What is the Geographical Focus of UNIGESTION SECONDARY VI Fund ?
20. What is the Target TVPI (Gross) of UNIGESTION SECONDARY VI Fund ?
10. Please indicate the headquarters and other offices of Unigestion.
"Financial Information 6. For which entity have you provided this financial data (IMC or parent)?"
"7. Which elements does yUnigestion's stewardship policy cover? This question has been adapted from the PRI. If you have a separate proxy voting policy document, you may still tick 'global policy around proxy voting' here. You may attach policy documents at the end of this questionnaire. Choices from: Global policy around proxy voting, Key stewardship objectives, Prioritisation approach of ESG factors and their link to engagement issues and targets, Prioritisation approach depending on entity (e.g. company or government), Specific approach to climate-related risks and opportunities, Stewardship tool usage across the organisation, including which, if any, tools are out of scope and when and how different tools are used and by whom, Stewardship tool usage across the organisation, including which, if any, tools are out of scope and when and how different tools are used and by whom, Approach to collaboration on stewardship, Escalation strategies, Conflicts of interest, Details on how the stewardship policy is implemented and which elements are mandatory, including how and when the policy can be overruled, How stewardship efforts and results should be communicated across the organisation to feed into investment decision-making and vice versa"
"""

system_msg = "You are a helpful IT-project that extracts information from documents and returns valid json data and json formatted data only."

# result = process_gpt(file_prompt, system_msg)
# print(result)
with open("questions_extracted.json", "w") as f:
    f.write(json.dumps(json.loads("[\n    \"Given Unigestion\u2019s stated commitment to ESG integration, how does the exclusion criteria based on the UN Principles for Responsible Investment influence their preferential access to off-market deals in secondary markets?\",\n    \"What is the location of Unigestion Head Office?\",\n    \"What is Unigestion full postal address?\",\n    \"Who clarifies Unigestion's goals (e.g., high returns, portfolio diversification, risk tolerance)?\",\n    \"How does Unigestion ensure consistency in valuations when market conditions fluctuate rapidly?\",\n    \"Unigestion highlights a 20+ year history in the global secondaries market. How has the acquisition of Akina in 2017 enhanced their positioning in mid-market opportunities, and what are the quantifiable impacts on fund performance metrics?\",\n    \"What metrics does Unigestion use to measure the alpha performance of its mid-market secondary investments?\",\n    \"How does the direct investment mindset in USEC VI ensure consistent due diligence across secondary funds, and how does this align with governance practices applied to other fund types?\",\n    \"How does Unigestion's commitment to the UN Principles for Responsible Investment influence its allocation strategy in USEC VI?\",\n    \"Which year was Unigestion established?\",\n    \"Have there been any corporate changes over the past five years at Unigestion? If yes, please provide details.\",\n    \"What is the investment period of UNIGESTION SECONDARY VI Fund?\",\n    \"Who has formal oversight and accountability for ESG / RI at Unigestion?\",\n    \"In which year was your responsible investment policy introduced?\",\n    \"Please describe, using a diagram, the organisational structure of Unigestion, highlighting the team that is responsible for Private Equity Secondaries and more specifically the team that would be responsible for the presented mandate. Please specify if this team is also covering other strategies.\",\n    \"If different from the GP, please provide the AIFM for UNIGESTION SECONDARY VI Fund.\",\n    \"Please provide an overview of the knowledge transfer services from Unigestion that would be available to this client.\",\n    \"What is the target of UNIGESTION SECONDARY VI Fund size in euros?\",\n    \"What is the anticipated number of underlying funds of UNIGESTION SECONDARY VI Fund?\",\n    \"What are Unigestion's long-term strategic objectives?\",\n    \"What is the final close date of UNIGESTION SECONDARY VI Fund?\",\n    \"What is the base currency of UNIGESTION SECONDARY VI Fund?\",\n    \"What is the target number of transactions of UNIGESTION SECONDARY VI Fund?\",\n    \"Please discuss Unigestion's approach to leverage and any challenges in this regard given current market conditions.\",\n    \"What is the target IRR (Gross) of UNIGESTION SECONDARY VI Fund?\",\n    \"Please outline Unigestion's investment strategy for this program and discuss how Unigestion's thinking in this space has evolved over time.\",\n    \"Please provide information on how Artificial Intelligence (if at all) is used in the process at Unigestion.\",\n    \"At what stage of funded commitments do you generally enter a secondary transaction?\",\n    \"Please describe Unigestion's investment process. Discuss their investment philosophy and rationale for investing in asset class, sourcing/research, fund/investment selection and portfolio construction.\",\n    \"Please provide a recent case-study example to demonstrate Unigestion's due diligence process to assess an underlying fund\u2019s ESG capabilities.\",\n    \"Please provide an example of a recent due diligence report from Unigestion.\",\n    \"Describe Unigestion's overall philosophy to ESG investing within private equity.\",\n    \"What is the typical transaction size for the proposed program of UNIGESTION SECONDARY VI Fund? Please also provide a historical range for past vehicles in the strategy.\",\n    \"Please describe how Unigestion assesses ESG policies and practices of underlying funds and how this is factored into the investment process including details of any third-party data used. If applicable, please provide a sample of Unigestion questionnaire for the underlying funds.\",\n    \"Please provide a list of Investment Committee members from Unigestion.\",\n    \"Please provide resumes for all Investment Committee members at Unigestion.\",\n    \"How are ESG risks and opportunities reported, considered, and documented by the ultimate decision-making body, such as the investment committee at Unigestion? If applicable, please also describe any look-through processes for underlying portfolios relating to both ESG and climate change.\",\n    \"Has Unigestion ever invested in, or approved for investment, a company which has had serious issues or required a termination? Please provide details. Did such an event result in any modification of the investment or due diligence process?\",\n    \"Please provide a current deal log at Unigestion. Company/fund names may be anonymised if required.\",\n    \"Please provide 2 examples of ESG related issues that have affected Unigestion's strategy, their impact and the activities undertaken in response.\",\n    \"Please discuss Unigestion's approach to transaction pricing and how this has changed over time. Please provide some views on the expected pricing environment during the life of the program.\",\n    \"Describe the potential risk factors that are specific to the proposed portfolio and specific safeguards contemplated to preserve investor capital at Unigestion.\",\n    \"Describe Unigestion's engagement process with underlying funds including who is responsible for this process. Please provide 2 examples of how you have engaged with underlying and the eventual outcome of this engagement.\",\n    \"How often and how soon after period end can Unigestion produce full audited reports?\",\n    \"Please describe the depth, frequency and timeliness of the performance reporting / portfolio overview that Unigestion provides to clients. Are in-house resources dedicated to this area? Please indicate the individuals that would be responsible for ongoing client service for this mandate.\",\n    \"Please provide details of Unigestion's ongoing monitoring of underlying funds and companies including how they often review their overall assessment of the underlying investment.\",\n    \"Have there been any adverse ESG events across the Unigestion portfolio, that have been communicated to investors? Please describe below.\",\n    \"Do Unigestion provide online reporting?\",\n    \"Please detail Unigestion's approach to exiting investments.\",\n    \"What is Unigestion's approach to capital recycling? Please describe any limits which are placed on the ability to recycle capital in this way (e.g. time limit, commitment cap etc.). Please also indicate the extent to which capital recycling has been used in prior funds/SMAs. What has been the impact on returns (IRR and MOIC) historically? What is the expected impact?\",\n    \"Are these individuals a centralised resource for Unigestion, asset class specific, or are they dedicated to ESG strategies?\",\n    \"Is the policy board-approved?\",\n    \"Does Unigestion employ any full-time professionals dedicated to ESG?\",\n    \"Describe Unigestion's approach to disclosing and following up on material ESG incidents to your investors.\",\n    \"Does Unigestion have a formal policy covering their approach to responsible investment?\",\n    \"What key indicators are published by Unigestion for this strategy at portfolio level?\",\n    \"Have Unigestion completed bfinance's firm-level ESG DDQ?\",\n    \"Please indicate the sectors Unigestion have decided to exclude at firm level.\",\n    \"How many percent is multi asset funds & mandates in Unigestion's asset?\",\n    \"If yes, please confirm your responses are up to date for this search?\",\n    \"If applicable what SFDR article is the proposed strategy categorised as?\",\n    \"Please indicate the management fee (in %) Unigestion will charge the client per annum based on a EUR 30m investment.\",\n    \"If applicable, can you detail the process engaged on an on-going basis to make sure the fund at Unigestion is fully compliant with the above?\",\n    \"Does the strategy apply any ESG-related exclusions (in addition to any exclusions that are applied at firm-level)?\",\n    \"Please list any other ESG/RI related collaborative organisations/initiatives Unigestion participate in.\",\n    \"Does Unigestion provide carbon reporting for this mandate?\",\n    \"How does Unigestion\u2019s ability to review 1,500 deals annually across Europe, North America, and Asia compare to their deal execution rate in 2022? How does this translate into their focus on mid-market alpha opportunities?\",\n    \"The management fee is based on what at Unigestion?\",\n    \"Has a potential investment ever been rejected solely by Unigestion due to ESG related issues? Please provide at least one example.\",\n    \"What is the maximum leverage allowed across fund levels in USEC VI, and how is this limitation justified against its stated risk strategy?\",\n    \"Do Unigestion explicitly consider environmental (including climate change) risks in the management of the portfolio?\",\n    \"The management fee at Unigestion is based on?\",\n    \"Who from Unigestion has built private equity teams?\",\n    \"Can 1000 deals be done per year if clients source unique opportunities from Unigestion's network of over 500 GPs?\",\n    \"Did de Dardel ever work in a company that has a business unit in agriculture business?\",\n    \"Describe the impact of ESG principles on fund performance metrics and allocation strategies as applied by Unigestion.\",\n    \"Please indicate the management fee (in %) Unigestion will charge the client per annum based on a EUR 30m investment.\",\n    \"What specific regulatory differences affect the structuring of Unigestion funds domiciled in Luxembourg versus those in Switzerland?\",\n    \"Post investment, how do Unigestion add value to their portfolio?\",\n    \"How does the investment approach for secondary funds (USEC VI) differ in structure and returns from the direct investment approach described in Unigestion\u2019s policies?\",\n    \"Describe the Fund Manager\u2019s philosophy to ensure appropriate oversight control of the portfolio at Unigestion.\",\n    \"Are there any managed accounts or otherwise that will invest alongside this program at Unigestion? If so, please provide some details on their allocation policy.\",\n    \"What is the anticipated number of underlying GPs of UNIGESTION SECONDARY VI Fund?\",\n    \"How does the Valuation Committee ensure consistency in applying IPEV guidelines for direct investments, and what happens in case of methodology changes?\",\n    \"Does Unigestion provide ESG-related reporting on the proposed fund/strategy?\",\n    \"Please provide details on Unigestion's experience working with European and Bank investors.\",\n    \"Please provide in EURm.\",\n    \"Please briefly explain Unigestion's experience in managing Private Equity Secondaries strategies. When did Unigestion first start managing strategies in this space?\",\n    \"How does Unigestion deal with any conflicts of interest both within the organisation and in relation to third parties?\",\n    \"Please make sure to provide all structures available of UNIGESTION SECONDARY VI Fund.\",\n    \"What is Unigestion's approach to incorporating ESG considerations into preparations for exit and ensuring continued positive impact in the post exit phase?\",\n    \"Limits (Sector, Geography)\",\n    \"What are the UNIGESTION SECONDARY VI Fund's Terms?\",\n    \"Please describe Unigestion ESG reporting capabilities for this fund, including frequency, metrics reported and level of transparency. Please also describe Unigestion's approach to ensuring data availability, quality and reliability.\",\n    \"Please discuss Unigestion's historical deal pipeline in terms of the conversion funds seen per annum, those taken through the various stages of diligence and those that closed.\",\n    \"Please provide details such as metrics, frequency etc. of Unigestion's climate related reporting including a sample report. Please comment on how you address areas where data is not available.\",\n    \"Is Unigestion currently a signatory to the Principles for Responsible Investment (PRI)?\",\n    \"If yes, when did Unigestion become a signatory?\",\n    \"How do you see Unigestion evolving in terms of ownership structure, client base, product offering, etc.?\",\n    \"Please detail all internal resourcing utilised for ESG analysis at Unigestion, with specific reference to dedicated in-house roles.\",\n    \"Please confirm that you do not use any leverage at the transaction level.\",\n    \"Does Unigestion take an active approach to winding up investments?\",\n    \"Please provide an example of an investment from Unigestion that they have reviewed and conducted due diligence on but that did not result in an investment.\",\n    \"Has Unigestion changed auditors in the past five years? If so, please explain why.\",\n    \"Are you aware of any current (or potential) changes to Unigestion's organisational structure or ownership?\",\n    \"Please make sure to provide all domiciles available of UNIGESTION SECONDARY VI Fund.\",\n    \"Have you set any organisation-wide targets on climate change?\",\n    \"Please provide a brief description and history of Unigestion.\",\n    \"Please provide information on the IT systems in place to support the investment process at Unigestion. Please make sure to identify the tools and systems that are proprietary / developed internally (if any).\",\n    \"Does the investment team at Unigestion actively engage with investee companies/underlying managers on ESG-related topics?\",\n    \"What is the target IRR (Net) of UNIGESTION SECONDARY VI Fund?\",\n    \"What is Unigestion's general approach to the use of leverage at both the company level and at the fund level?\",\n    \"What is the target TVPI (Net) of UNIGESTION SECONDARY VI Fund?\",\n    \"What specific attributes make USEC VI\u2019s portfolio of small secondary transactions \u2018unlikely to be replicated elsewhere,\u2019 and how does this impact Unigestion\u2019s ability to maintain consistent alpha performance across cycles?\",\n    \"Please briefly explain Unigestion's experience in managing Private Equity strategies. When did the firm first begin investing in the asset class?\",\n    \"Are any external resources or organisations used for ESG analysis at Unigestion?\",\n    \"If applicable, please list any specific ESG-related KPIs measured and/or SDG alignment within the strategy?\",\n    \"Does the investment team at Unigestion explicitly consider ESG / RI as part of the investment process of the proposed strategy at Unigestion?\",\n    \"What is the main activities of Unigestion?\",\n    \"What are the key qualitative and quantitative criteria addressed during due diligence at Unigestion?\",\n    \"Do you have a climate-related investment policy?\",\n    \"Please provide the details of the ownership structure at Unigestion, i.e. details of shareholders with % ownership for each individual or group.\",\n    \"Describe Unigestion's valuation process including the name and the role of any independent, external valuers. With what frequency and according to which accounting standards are investments valued? How frequently do Unigestion receive valuations from their investments?\",\n    \"Please describe the investment committee process and the voting procedures at Unigestion. Are investment decisions made unanimously / through majority (specify %) at Unigestion? Does any member of the Investment Committee have a veto right at Unigestion?\",\n    \"Please describe Unigestion's due diligence process for the strategy, from initial screening to final investment decision.\",\n    \"What is the fundraising start date of UNIGESTION SECONDARY VI Fund?\",\n    \"How are investments sourced and selected at Unigestion and how is this approach differentiated to Unigestion's competitors?\",\n    \"If yes, please list these additional exclusions.\",\n    \"What is the minimum ticket size in euros of UNIGESTION SECONDARY VI Fund?\",\n    \"How does Unigestion's strategy for small, non-auctioned transactions mitigate competition risk in the mid-market?\",\n    \"If Unigestion targets 328 primary, 353 secondary, and 350 direct opportunities annually, why is the mid-market segment particularly advantageous for secondary investments, as stated in the USEC VI fund strategy?\",\n    \"How has the acquisition of Akina in 2017 influenced Unigestion\u2019s ability to scale its private equity strategies?\",\n    \"What is the next close date of UNIGESTION SECONDARY VI Fund?\",\n    \"What is the hard cap of UNIGESTION SECONDARY VI Fund size in euros?\",\n    \"State the number deployed per year / months by UNIGESTION SECONDARY VI Fund?\",\n    \"With which entity would the client be contracting at Unigestion?\",\n    \"Please detail the proposed portfolio\u2019s investment strategy. Where do you see the most attractive opportunities in the current market environment? How does this fit with the proposed portfolio construction? Are there any strategies you would exclude?\",\n    \"Will you be using subscription line financing at Unigestion? If so, what is the purpose and intention of using the line and what are the terms? Provide details including % of commitments. What has been the impact on returns (IRR and MOIC) historically? What is the expected impact? How will you report on this to LPs?\",\n    \"What is the amount raised so far for UNIGESTION SECONDARY VI Fund size in euros?\",\n    \"Does the proposed mandate fall into scope of achieving net zero at Unigestion?\",\n    \"What is Unigestion entity to be invoiced?\",\n    \"What is the geographical focus of UNIGESTION SECONDARY VI Fund?\",\n    \"What is the target TVPI (Gross) of UNIGESTION SECONDARY VI Fund?\",\n    \"Please indicate the headquarters and other offices of Unigestion.\",\n    \"For which entity have you provided this financial data?\",\n    \"Which elements does Unigestion's stewardship policy cover?\"\n]")))

[
    "Given Unigestion’s stated commitment to ESG integration, how does the exclusion criteria based on the UN Principles for Responsible Investment influence their preferential access to off-market deals in secondary markets?",
    "What is the location of Unigestion Head Office?",
    "What is Unigestion full postal address?",
    "Who clarifies Unigestion's goals (e.g., high returns, portfolio diversification, risk tolerance)?",
    "How does Unigestion ensure consistency in valuations when market conditions fluctuate rapidly?",
    "Unigestion highlights a 20+ year history in the global secondaries market. How has the acquisition of Akina in 2017 enhanced their positioning in mid-market opportunities, and what are the quantifiable impacts on fund performance metrics?",
    "What metrics does Unigestion use to measure the alpha performance of its mid-market secondary investments?",
    "How does the direct investment mindset in USEC VI ensure consistent due diligence across seconda